In [1]:
import os
import glob
import time
from datetime import timedelta
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt



Matplotlib is building the font cache; this may take a moment.


### Explanation of the `load_data` Function

This function performs a series of data preprocessing steps to clean and prepare crash and vehicle datasets for analysis. Below is a detailed breakdown of the operations:

---

#### **Function Purpose**
- **Inputs**:
  - `crash_data_path`: Path to the crashes dataset (CSV file).
  - `vehicles_data_path`: Path to the vehicles dataset (CSV file).
- **Output**:
  - A cleaned DataFrame filtered for the years 2022-2023.

---

#### **Step-by-Step Breakdown**

1. **Loading Datasets**:
   - The crashes dataset is loaded into a DataFrame (`crashes_df`), and its shape is displayed.
   - The vehicles dataset is loaded into a DataFrame (`vehicles_df`), and its shape is displayed.

2. **Merging Datasets**:
   - The two datasets are merged on the common column `COLLISION_ID` using a left join, resulting in `NYC_crashes_df`.
   - Initial information about the merged dataset is printed (shape, first few rows, and summary).

3. **Dropping Unnecessary Columns**:
   - Columns such as `ZIP CODE`, `LOCATION`, and contributing factor columns are dropped to reduce data complexity.

4. **Creating and Adjusting Time Features**:
   - A new column `YEAR` is created by extracting the year from the `CRASH_DATE` column.
   - `CRASH_DATE` and `CRASH_TIME` columns are dropped, as they are no longer needed.

5. **Filtering for Specific Years**:
   - The dataset is filtered to include only rows corresponding to the years 2022 and 2023.

6. **Removing Duplicates**:
   - Duplicate rows are removed, and the shape before and after this operation is printed.

7. **Handling Missing Values**:
   - Missing values in **text columns** (e.g., `BOROUGH`, `VEHICLE TYPE CODE`) are replaced with `'unknown'`.
   - Missing values in **numeric columns** (e.g., `LATITUDE`, `NUMBER OF PERSONS INJURED`) are replaced with `0`.
   - Remaining null counts per column are checked and displayed.

8. **Final Output**:
   - The cleaned DataFrame is returned, ready for analysis.

---

#### **Key Insights**
- The function ensures the data is well-prepared by addressing:
  - Duplicates and missing values.
  - Irrelevant columns for the analysis.
  - Filtering only relevant timeframes.
- Missing values are systematically handled, improving the robustness of subsequent analyses or modeling steps.

In [2]:
def load_data(crash_data_path, vehicles_data_path):
    """
    Function to load the crashes and vehicles datasets, merge them on COLLISION_ID,
    create a YEAR column, filter data for 2022-2023, handle duplicates, and manage missing values.

    Parameters:
    crash_data_path (str): Path to the crashes dataset file.
    vehicles_data_path (str): Path to the vehicles dataset file.

    Returns:
    DataFrame: The cleaned DataFrame for years 2022-2023.
    """
    # Load the crashes dataset
    print("******************Loading crash data**************************\n")
    crashes_df = pd.read_csv(crash_data_path)
    print("Crashes DataFrame Loaded. Shape of crashes_df:", crashes_df.shape)

    # Load the vehicles dataset
    print("******************Loading vehicles data**************************\n")
    vehicles_df = pd.read_csv(vehicles_data_path)
    print("Vehicles DataFrame Loaded. Shape of vehicles_df:", vehicles_df.shape)
    

    # Merge crashes and vehicles datasets on COLLISION_ID
    print("******************Merging data**************************\n")
    NYC_crashes_df = vehicles_df.merge(crashes_df, on='COLLISION_ID', how='left')
    
    print("\n*************** Data Summary Before Cleaning **************\n")
    print("Shape of merged DataFrame (NYC_crashes_df):", NYC_crashes_df.shape, "\n")
    print(NYC_crashes_df.head(3), "\n")
    print(NYC_crashes_df.info(), "\n")
    
    # Drop specified columns
    columns_to_drop = ['ZIP CODE', 'LOCATION', 'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2']
    NYC_crashes_df = NYC_crashes_df.drop(columns=columns_to_drop)
    print(f"\nDropped columns: {columns_to_drop}\n")
    print("Shape after dropping columns:", NYC_crashes_df.shape)

    # Drop duplicates
    print("\n*************** Removing Duplicates **************\n")
    initial_shape = NYC_crashes_df.shape
    NYC_crashes_df = NYC_crashes_df.drop_duplicates()
    print(f"Removed duplicates. Shape changed from {initial_shape} to {NYC_crashes_df.shape}")

    # Handle missing values
    print("\n*************** Managing Missing Values **************\n")
    print(NYC_crashes_df.isnull().sum())

    # Replace missing values in text columns with 'unknown'
    text_columns = ['BOROUGH','ON STREET NAME', 'PRE_CRASH', 'POINT_OF_IMPACT', 
                    'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 
                    'CONTRIBUTING FACTOR VEHICLE 3', 'VEHICLE TYPE CODE 1','VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3',
                   'TRAVEL_DIRECTION','PRE_CRASH','POINT_OF_IMPACT', 'VEHICLE_DAMAGE','VEHICLE_DAMAGE_1', 'PUBLIC_PROPERTY_DAMAGE','PUBLIC_PROPERTY_DAMAGE_TYPE']
    for col in text_columns:
        if col in NYC_crashes_df.columns:
            NYC_crashes_df[col] = NYC_crashes_df[col].fillna('unknown')
    print(f"Replaced missing values in text columns with 'unknown'.")

    # Replace missing values in numeric columns with 0
    numeric_columns = ['LATITUDE', 'LONGITUDE','NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'VEHICLE_OCCUPANTS']
    for col in numeric_columns:
        if col in NYC_crashes_df.columns:
            NYC_crashes_df[col] = NYC_crashes_df[col].fillna(0)
    print(f"Replaced missing values in numeric columns with 0.")
 
    # Check for remaining nulls
    print("\n*************** Checking for Remaining Nulls **************\n")
    null_counts = NYC_crashes_df.isnull().sum()
    print("Null counts per column (after cleaning):")
    print(null_counts)
    
      

    return NYC_crashes_df

### Explanation of the `preprocess_data` Function

This function preprocesses the NYC crash dataset to clean, derive new features, and prepare the data for machine learning models. It transforms raw crash data into a structured and feature-enriched format, making it suitable for analytical and predictive tasks.

---

#### **Function Purpose**
- **Input**:
  - `df`: Raw crash dataset (Pandas DataFrame).
- **Outputs**:
  - A cleaned and feature-enriched DataFrame.
  - A preprocessing pipeline ready for ML models.

---

#### **Step-by-Step Breakdown**

1. **Creating Target Label (`CRASH_SEVERITY`)**:
   - **Purpose**: Classify crash severity based on fatalities, injuries, or property damage.
   - **Implementation**:
     - Rows with fatalities (`NUMBER OF PERSONS KILLED`), injuries (`NUMBER OF PERSONS INJURED`), or public property damage (`PUBLIC_PROPERTY_DAMAGE` = 'Y') are labeled as `High Impact`.
     - All other rows are labeled as `Low Impact`.
   - **Output**: A new column `CRASH_SEVERITY` is added, and the class distribution is printed.

2. **Handling Date and Time Columns**:
   - **Purpose**: Extract date and time features for deeper temporal analysis.
   - **Steps**:
     - Convert `CRASH DATE` and `CRASH TIME` into datetime format.
     - Extract features like `year`, `month`, `day_of_week`, and `hour`.
     - Create a binary column `is_weekend` indicating whether the crash occurred on a weekend.

3. **Deriving `location_type`**:
   - **Purpose**: Identify whether the crash occurred in an urban area or on a highway.
   - **Logic**:
     - If `BOROUGH` is known and valid, it is classified as `Urban`.
     - Otherwise, it is classified as `Highway`.

4. **Calculating Total Vehicles Involved**:
   - **Purpose**: Summarize the number of vehicles involved in each crash.
   - **Implementation**:
     - Count the non-missing and non-`unknown` values in vehicle-related columns (`VEHICLE TYPE CODE 1`, `VEHICLE TYPE CODE 2`, `VEHICLE TYPE CODE 3`).

5. **Categorizing Time of Day**:
   - **Purpose**: Add a temporal dimension by categorizing crashes into time-of-day segments.
   - **Categories**:
     - `Morning` (5 AM–12 PM)
     - `Afternoon` (12 PM–5 PM)
     - `Evening` (5 PM–9 PM)
     - `Night` (9 PM–5 AM)
     - `Unknown` for missing hours.

6. **Dropping Irrelevant Columns**:
   - **Purpose**: Reduce dataset complexity and redundancy.
   - **Implementation**:
     - Columns related to unique IDs, collision identifiers, and detailed crash metrics are dropped.
     - This ensures a cleaner dataset focused on derived and essential features.

7. **Final Output**:
   - **Result**:
     - The cleaned DataFrame is returned with derived features such as `CRASH_SEVERITY`, `location_type`, `total_vehicles_involved`, and `time_of_day`.

---

#### **Key Insights**
- This preprocessing pipeline transforms raw data into actionable insights by adding features and normalizing values.
- The dataset is structured for machine learning tasks with a focus on crash severity prediction.
- Temporal and spatial attributes enrich the dataset, enabling robust analyses.

In [3]:
def preprocess_data(df):
    """
    Preprocess the NYC crash dataset to clean, derive new features, normalize text, 
    prepare text columns for ML models using TF-IDF and One-Hot Encoding, and define crash severity.
    
    Parameters:
    df (DataFrame): The raw crash dataset.
    
    Returns:
    DataFrame: The cleaned and feature-enriched DataFrame.
    ColumnTransformer: Preprocessing pipeline for use in ML models.
    """
    print("*************Creating Target Label*****************\n")
    # Step 1: Define target label "CRASH_SEVERITY"
    def classify_severity(row):
        if row['NUMBER OF PERSONS KILLED'] >= 1 or row['PUBLIC_PROPERTY_DAMAGE'] == 'Y' or row['NUMBER OF PERSONS INJURED'] >= 1:
            return 'High Impact'  # Fatality present or property damage or injury
        else:
            return 'Low Impact'  # No injuries, no fatalities, no property damage
    
    # Step 2: Apply the function to classify severity
    df['CRASH_SEVERITY'] = df.apply(classify_severity, axis=1)
    
    # Calculate class distribution
    class_distribution = df['CRASH_SEVERITY'].value_counts()

    print("Class Distribution:\n", class_distribution)

    # Step 3: Convert CRASH DATE and CRASH TIME to datetime formats
    df['CRASH_DATE'] = pd.to_datetime(df['CRASH_DATE'], errors='coerce')
    df['CRASH_TIME'] = pd.to_datetime(df['CRASH_TIME'], format='%H:%M', errors='coerce')
    
    # Step 4: Extract date and time features
    print("*************Extracting date and time features*****************\n")
    df['year'] = df['CRASH_DATE'].dt.year
    df['month'] = df['CRASH_DATE'].dt.month
    df['day_of_week'] = df['CRASH_DATE'].dt.day_name()
    df['hour'] = df['CRASH_TIME'].dt.hour
    df['is_weekend'] = df['day_of_week'].isin(['Saturday', 'Sunday']).astype(int)
    
    # Step 5: Derive 'location_type'
    print("*************Deriving 'Location type' feature*****************\n")
    def location_type(row):
        if row['BOROUGH'] and row['BOROUGH'].lower() != 'unknown':
            return 'Urban'
        else:
            return 'Highway'
    df['location_type'] = df.apply(location_type, axis=1)
    
    # Step 6: Count total vehicles involved
    print("*************Calculating Number of Vehicles involved in the Crash*****************\n")
    vehicle_columns = ['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3']
    df['total_vehicles_involved'] = df[vehicle_columns].apply(lambda row: row.str.lower().ne('unknown').sum(), axis=1)
    
    
    # Step 7: Categorize time of day
    print("*************Categorize time of day********************\n")
    def time_of_day(hour):
        if pd.isna(hour):  # Handle missing hour
            return 'Unknown'
        if 5 <= hour < 12:
            return 'Morning'
        elif 12 <= hour < 17:
            return 'Afternoon'
        elif 17 <= hour < 21:
            return 'Evening'
        else:
            return 'Night'
    df['time_of_day'] = df['hour'].apply(time_of_day)
    

    # Step 9: Drop columns
    columns_to_drop = ['NUMBER OF PERSONS KILLED', 'NUMBER OF PERSONS INJURED', 
                   'PUBLIC_PROPERTY_DAMAGE', 'PUBLIC_PROPERTY_DAMAGE_TYPE',
                   'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 
                   'UNIQUE_ID', 'COLLISION_ID']
    df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
    
    print(f"\n processed data :  {df.head(2)} \n ")

    return df 

### Explanation of the `split_data` Function

This function preprocesses the dataset by performing feature transformation and splitting it into training, validation, and test sets. It ensures that the data is ready for machine learning model training and evaluation, using appropriate encoding and scaling techniques for different types of features.

---

#### **Function Purpose**
- **Input**:
  - `df`: Preprocessed crash dataset (Pandas DataFrame).
- **Outputs**:
  - Split and processed features and target data for training, validation, and testing sets (`X_train`, `X_val`, `X_test`, `y_train`, `y_val`, `y_test`).

---

#### **Step-by-Step Breakdown**

1. **Splitting Data by Year**:
   - **Purpose**: Separate the data into train (2022) and test (2023) sets based on the `year` column.
   - **Implementation**:
     - `train_data`: All records from 2022.
     - `test_data`: All records from 2023.
   - **Output**: The dataset is divided into training and test sets for model training and evaluation.

2. **Selecting Features and Target**:
   - **Purpose**: Define the features and target variable for model training.
   - **Implementation**:
     - Features (`train_features`) include temporal, spatial, and vehicle-related attributes.
     - The target (`y_train_full`) is the crash severity (`CRASH_SEVERITY`).

3. **Identifying Categorical and Numeric Columns**:
   - **Purpose**: Identify which features need to undergo specific transformations for machine learning models.
   - **Categorical columns**: Includes features like `location_type`, `time_of_day`, `TRAVEL_DIRECTION`, etc.
   - **Numeric columns**: Includes features like `total_vehicles_involved`, `hour`, and `VEHICLE_OCCUPANTS`.

4. **Transforming Categorical Features with One-Hot Encoding**:
   - **Purpose**: Convert categorical features into a format suitable for machine learning models.
   - **Implementation**:
     - Use `OneHotEncoder` to perform one-hot encoding on the categorical columns, with `drop='first'` to avoid multicollinearity.
   - **Output**: A matrix of one-hot encoded features for categorical variables.

5. **Scaling Numeric Features**:
   - **Purpose**: Standardize numeric features to have a mean of 0 and standard deviation of 1.
   - **Implementation**:
     - Use `StandardScaler` to scale the numeric columns.
   - **Output**: A scaled matrix of numeric features.

6. **Concatenating Transformed Features**:
   - **Purpose**: Combine the transformed categorical and numeric features into a single DataFrame.
   - **Implementation**:
     - The one-hot encoded categorical features are combined with the scaled numeric features using `pd.concat`.
     - Column names are ensured to be consistent by converting all column names to strings.
   - **Output**: A processed DataFrame (`X_processed`) containing all features in the correct format for machine learning.

7. **Splitting Data into Training, Validation, and Test Sets**:
   - **Purpose**: Split the data into three subsets: training, validation, and test.
   - **Implementation**:
     - Use `train_test_split` to first create an 80% training and 20% temporary set.
     - Split the temporary set into 50% validation and 50% test, ensuring stratified sampling based on the target variable (`CRASH_SEVERITY`).
   - **Output**: The final split data: `X_train`, `X_val`, `X_test`, `y_train`, `y_val`, and `y_test`.

---

#### **Key Insights**
- **Stratified Sampling**: Ensures that the distribution of crash severity (`CRASH_SEVERITY`) is similar across the training, validation, and test sets, which is critical for balanced model performance.
- **One-Hot Encoding and Scaling**: Prepares the data for machine learning models by transforming categorical variables and scaling numeric features.
- **Feature Engineering**: Combines various features (temporal, spatial, vehicle-related) to create a rich set of input features for model training.

--- 

#### **Final Output**
- The function returns the training, validation, and test sets for both features (`X_train`, `X_val`, `X_test`) and target labels (`y_train`, `y_val`, `y_test`), ready for model development and evaluation.


In [4]:
def split_data(df):
    """
    Preprocess the data and split into train, validation, and test sets.
    """
    # Split train (2022) and test (2023) data
    train_data = df[df['year'] == 2022]
    test_data = df[df['year'] == 2023]
    
    print(df.dtypes)

    # Train features and target
    train_features = ['location_type', 'total_vehicles_involved', 'time_of_day', 'is_weekend',
                      'hour', 'day_of_week', 'month', 'year',
                      'CONTRIBUTING FACTOR VEHICLE 1',
                      'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
                      'VEHICLE_DAMAGE', 'VEHICLE_DAMAGE_1', 'TRAVEL_DIRECTION', 'VEHICLE_OCCUPANTS', 'PRE_CRASH']
    
    X_train_full = train_data[train_features]
    y_train_full = train_data['CRASH_SEVERITY']

    # Step 1: Identify columns
    categorical_columns = ['location_type', 'is_weekend', 'day_of_week', 'month', 'TRAVEL_DIRECTION', 'time_of_day',
                          'CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
                    'VEHICLE_DAMAGE', 'VEHICLE_DAMAGE_1', 'PRE_CRASH']
    numeric_columns = ['total_vehicles_involved', 'hour', 'VEHICLE_OCCUPANTS']

    # Step 2: One-hot encoding for categorical columns
    categorical_transformer = OneHotEncoder(drop='first', sparse_output=False)
    X_cat = categorical_transformer.fit_transform(X_train_full[categorical_columns])

    
    # Step 4: Standard scaling for numeric columns
    numeric_transformer = StandardScaler()
    X_num = numeric_transformer.fit_transform(X_train_full[numeric_columns])

    # Step 5: Concatenate all transformed features
    X_cat_df = pd.DataFrame(X_cat, columns=categorical_transformer.get_feature_names_out(categorical_columns))
    X_num_df = pd.DataFrame(X_num, columns=numeric_columns)

    # Step 6: Concatenate all transformed columns into a single DataFrame
    X_processed = pd.concat([X_cat_df, X_num_df], axis=1)

    # Ensure all column names are strings (optional but useful for consistency)
    X_processed.columns = X_processed.columns.astype(str)

    # Check the transformed data
    print(X_processed.head())

    # Split the data into training, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X_processed, y_train_full, test_size=0.2, random_state=5, stratify=y_train_full)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=5, stratify=y_temp)
    
    
    return X_train, X_val, X_test, y_train, y_val, y_test

## 1. `evaluate_model` Function
This function computes and displays a confusion matrix for evaluating classification models. It helps visualize the model's performance by comparing predicted vs. actual class labels.

- **Parameters**:
  - `y_pred`: The predicted class labels from the model.
  - `y_true`: The actual class labels.
  - `class_names` (optional): Names of classes for better labeling.
  
- **Returns**:
  - The confusion matrix as a NumPy array.

---

## 2. `fit_model` Function
This function trains and evaluates multiple classification models (Decision Tree, Random Forest) with hyperparameter optimization. It uses `GridSearchCV` for tuning and measures performance metrics such as accuracy, precision, recall, and F1 score.

### Steps:
1. **Decision Tree Model**:
   - Hyperparameter tuning using `GridSearchCV` with parameters like `max_depth` and `min_samples_split`.
   - Validation metrics (accuracy, precision, recall, F1 score) are calculated and stored.
  
2. **Random Forest Model**:
   - Optimized hyperparameter tuning with fewer combinations and 3-fold cross-validation to reduce runtime.
   - Uses `n_jobs=-1` for parallel processing.
   - Validation metrics are calculated and stored.

3. **Model Evaluation**:
   - For each model, the function:
     - Prints key validation metrics.
     - Displays the classification report (detailed performance per class).
     - Generates and displays the confusion matrix using the `evaluate_model` function.

---

### **Parameters**:
- `X_train`, `y_train`: Training dataset.
- `X_val`, `y_val`: Validation dataset.

### **Returns**:
A dictionary containing:
- Fitted models (`Decision Tree` and `Random Forest`).
- Their validation metrics (accuracy, precision, recall, F1 score).
- Predictions on the validation set.

---

## Key Features and Optimizations

### **1. Hyperparameter Tuning**:
- Decision Tree and Random Forest models are tuned using `GridSearchCV` with carefully selected parameter ranges.
- Reduced parameter combinations and cross-validation folds for faster execution.
   
### **2. Performance Metrics**:
- **Accuracy**: Measures overall correctness.
- **Precision, Recall, F1 Score**: Evaluate specific aspects of classification performance.

### **3. Parallel Processing**:
- For Random Forest, the function leverages parallel processing (`n_jobs=-1`) for quicker hyperparameter tuning.

### **4. Confusion Matrix and Reports**:
- The `evaluate_model` function visualizes performance and provides insights into class-wise predictions.
- The classification report summarizes precision, recall, and F1 scores for each class.

### **5. Modularity**:
- Each model is trained, validated, and evaluated independently, making the function modular and extensible.

---

## Purpose
This function is ideal for testing multiple models in a streamlined manner while evaluating their performance on validation data. It balances thoroughness (via hyperparameter tuning) with efficiency


In [5]:
def evaluate_model(y_pred, y_true, class_names=None):
    """
    Generates and displays a confusion matrix with a heatmap for better visualization.
    
    Parameters:
    y_pred (array-like): Predicted labels from the model.
    y_true (array-like): Actual labels from the data.
    class_names (list, optional): List of class names for labeling the matrix.
    
    Returns:
    cm (ndarray): The confusion matrix.
    """
    # Calculate the confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    print("Confusion Matrix:\n", cm)
    return cm

def fit_model(X_train, y_train, X_val, y_val):
    """
    Fits a Decision Tree, Random Forest, and Gradient Boosting model sequentially 
    to the input data, and applies hyperparameter tuning to improve recall.

    This function takes in the feature matrix (X), the target variable (y), 
    builds the models, tunes them, and returns the fitted models along with their 
    performance metrics on the validation set.

    Args:
        X_train (pd.DataFrame): Feature matrix for training.
        y_train (pd.Series): Target variable for training.
        X_val (pd.DataFrame): Feature matrix for validation.
        y_val (pd.Series): Target variable for validation.

    Returns:
        dict: A dictionary containing the fitted models and their respective performance metrics.
    """
    
    models_results = {}
    print("\n ******************running decision tree model*************************\n")
    # Decision Tree Model
    dt_params = {'max_depth': [3, 5, 10, None], 'min_samples_split': [2, 5, 10]}
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_grid_search = GridSearchCV(dt_model, dt_params, scoring='recall_weighted', cv=5)
    dt_grid_search.fit(X_train, y_train)
    dt_best_model = dt_grid_search.best_estimator_
    
    # Validate the Decision Tree model
    dt_y_val_pred = dt_best_model.predict(X_val)
    dt_accuracy = accuracy_score(y_val, dt_y_val_pred)
    dt_precision = precision_score(y_val, dt_y_val_pred, average='weighted')
    dt_recall = recall_score(y_val, dt_y_val_pred, average='weighted')
    dt_f1 = f1_score(y_val, dt_y_val_pred, average='weighted')

    # Store results
    models_results['Decision Tree'] = {
        'model': dt_best_model, 'accuracy': dt_accuracy, 'precision': dt_precision,
        'recall': dt_recall, 'f1': dt_f1, 'y_val_pred': dt_y_val_pred
    }
    print("\n ******************running Random Forest model*************************\n")
    # Optimized Random Forest hyperparameters
    rf_params = {
        'n_estimators': [50, 100],  # Fewer estimators to speed up training
        'max_depth': [5, 10],  # Limited max depth
        'min_samples_split': [2, 5]  # Fewer combinations
    }

    # Initialize the Random Forest model
    rf_model = RandomForestClassifier(random_state=42)

    # GridSearchCV with fewer cross-validation folds and parallel processing
    rf_grid_search = GridSearchCV(
        rf_model, rf_params, scoring='recall_weighted', cv=3, n_jobs=-1  # Use 3-fold CV and parallel processing
    )

    # Fit the model
    rf_grid_search.fit(X_train, y_train)
    rf_best_model = rf_grid_search.best_estimator_

    # Validate the Random Forest model
    rf_y_val_pred = rf_best_model.predict(X_val)
    rf_accuracy = accuracy_score(y_val, rf_y_val_pred)
    rf_precision = precision_score(y_val, rf_y_val_pred, average='weighted')
    rf_recall = recall_score(y_val, rf_y_val_pred, average='weighted')
    rf_f1 = f1_score(y_val, rf_y_val_pred, average='weighted')

    # Store results
    models_results['Random Forest'] = {
        'model': rf_best_model, 'accuracy': rf_accuracy, 'precision': rf_precision,
        'recall': rf_recall, 'f1': rf_f1, 'y_val_pred': rf_y_val_pred
    }
   

    print("\n ***************Display classification report and confusion matrix for each model*****************\n")
    # Display classification report and confusion matrix for each model
    for model_name, model_data in models_results.items():
        print(f"\nModel: {model_name}")
        print(f"Validation Accuracy: {model_data['accuracy']:.4f}")
        print(f"Precision: {model_data['precision']:.4f}")
        print(f"Recall: {model_data['recall']:.4f}")
        print(f"F1 Score: {model_data['f1']:.4f}\n")
        print("\nClassification Report:")
        print(classification_report(y_val, model_data['y_val_pred']))

        # Confusion Matrix
        print("\nConfusion Matrix:")
        cm = evaluate_model(model_data['y_val_pred'], y_val)
    
    return models_results

### Running the main Function

In [6]:
def main():
    # Step 1: Load the data using the `load_data` function
    crash_data_path = "crashes_data.csv" 
    vehicles_data_path = "vehicles_data.csv"
    
    # Load and clean the data
    print("\n*************** Loading Data ***************\n")
    NYC_crashes_df = load_data(crash_data_path, vehicles_data_path)
    print("\n*************** Loaded and Cleaned Data ***************\n")
    print(NYC_crashes_df.head(3))
    print(f"Shape of the cleaned DataFrame: {NYC_crashes_df.shape}\n")
    
    # Step 2: Preprocess the data
    print("\n*************** Preprocessing Data ***************\n")
    NYC_crashes_df = preprocess_data(NYC_crashes_df)
    if NYC_crashes_df is None:
        raise ValueError("Data preprocessing failed. The returned DataFrame is None.")
    print("Preprocessed data shape: ", NYC_crashes_df.shape)
    
    # Step 3: Split the data
    print("\n*************** Splitting Data ***************\n")
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(NYC_crashes_df)
        
    # Step 4: Fit and evaluate the model
    print("\n*************** Fitting and Evaluating Models ***************\n")
    models_results = fit_model(X_train, y_train, X_val, y_val)
    

In [7]:
# Run the main function
if __name__ == "__main__":
    main()


*************** Loading Data ***************

******************Loading crash data**************************

Crashes DataFrame Loaded. Shape of crashes_df: (200489, 16)
******************Loading vehicles data**************************

Vehicles DataFrame Loaded. Shape of vehicles_df: (405570, 15)
******************Merging data**************************


*************** Data Summary Before Cleaning **************

Shape of merged DataFrame (NYC_crashes_df): (405570, 30) 

   UNIQUE_ID  COLLISION_ID  CRASH_DATE CRASH_TIME TRAVEL_DIRECTION  \
0   20404266       4612832  03/09/2023      20:38             West   
1   20393371       4607823  02/20/2023       7:00            North   
2   20338184       4582398  11/08/2022      19:03            North   

   VEHICLE_OCCUPANTS             PRE_CRASH            POINT_OF_IMPACT  \
0                1.0                Parked           Center Front End   
1                1.0    Stopped in Traffic  Right Front Quarter Panel   
2                1.0 

# Model Review and Analysis

### Objective:
The model is designed to predict crash severity (either "Low Impact" or "High Impact") based on various features, including the date and time of the crash, the contributing factors, and the number of vehicles involved.

### Key Steps in the Process:

#### 1. **Data Preprocessing:**
   - The data was filtered to include only crashes from 2022 and 2023, resulting in a cleaned dataset with 405,570 records.
   - Missing values were appropriately handled by filling missing text columns with "unknown" and numeric columns with 0.
   - Unnecessary columns such as "ZIP CODE" and "LOCATION" were dropped to streamline the dataset.
   - Additional columns like `YEAR` were derived from the `CRASH_DATE`, and new features like `location_type` and `total_vehicles_involved` were created.

#### 2. **Feature Engineering:**
   - Date and time-related features were derived, such as `year`, `month`, `day_of_week`, and `hour`, which may help in identifying patterns based on the time of day, day of the week, or month.
   - A new feature `is_weekend` was added to capture whether the crash occurred on a weekend, which could be important for understanding crash frequency.
   - The number of vehicles involved was calculated, which likely correlates with the severity of the crash.

#### 3. **Handling Categorical Features:**
   - Several categorical features,were encoded as dummy variables, allowing the model to process them effectively.

---

# Model Performance Summary

The results of the Decision Tree and Random Forest models were evaluated using validation accuracy, precision, recall, F1 score, classification reports, and confusion matrices. Here’s a summary of the outcomes:

---

## Decision Tree
- **Validation Accuracy:** 70.51%
- **Precision:** 69.59%
- **Recall:** 70.51%
- **F1 Score:** 69.51%

### Classification Report:
- The Decision Tree model showed better performance on the **"Low Impact"** class compared to the **"High Impact"** class.
  - For **High Impact**, the precision was **63%**, with a recall of **48%**, indicating it struggles to correctly identify instances of this class.
  - For **Low Impact**, the precision was **74%**, with a recall of **83%**, demonstrating stronger predictive power for this class.
- The overall weighted averages for precision, recall, and F1 score were around **70%**, reflecting balanced but slightly skewed performance.

### Confusion Matrix:
- **True Positives (High Impact):** 3711  
- **False Positives (High Impact):** 3995  
- **True Positives (Low Impact):** 11140  
- **False Positives (Low Impact):** 2215  

---

## Random Forest
- **Validation Accuracy:** 69.53%
- **Precision:** 70.36%
- **Recall:** 69.53%
- **F1 Score:** 64.86%

### Classification Report:
- The Random Forest model also demonstrated better performance on the **"Low Impact"** class:
  - For **High Impact**, precision was **73%**, but recall was significantly lower at **27%**, indicating a high rate of missed classifications.
  - For **Low Impact**, precision was **69%**, with a high recall of **94%**, showing strong performance in capturing this class.
- The weighted average F1 score was **64.86%**, slightly lower than the Decision Tree model, largely due to imbalanced performance across classes.

### Confusion Matrix:
- **True Positives (High Impact):** 2068  
- **False Positives (High Impact):** 5638  
- **True Positives (Low Impact):** 12576  
- **False Positives (Low Impact):** 779  

---

## Key Insights
1. Both models performed better at identifying **"Low Impact"** instances than **"High Impact"**, suggesting class imbalance or inherent difficulty in distinguishing "High Impact."
2. The Decision Tree had slightly higher F1 scores for overall performance, particularly benefiting from more balanced recall across classes.
3. The Random Forest, while providing better precision for the "High Impact" class, struggled with recall for this class, resulting in a large number of misclassifications.
4. Future improvements could focus on addressing class imbalance or improving feature engineering to boost performance for **"High Impact"** cases.
